One of the main issue you can get is to use the wrong variable. If you defined two variables with the same name in different places you may think you are going to use the first one but no, you are using the second one. It is not easy to debug that kind of error since the program runs until the end most of time (but the result is wrong).

In [1]:
def g(x):
    a = 2
    print(a,b,x)

What does it mean? How can we print b if not defined? The answer is b must be defined somewhere else and must be visible by g().

The rule is **use the more local variable** meaning 

* the one defined in the function
* if none, the one define in the function which called the function
* if none idem until you reach the main program

In [2]:
def g(x):
    a = 2        # this is a new a
    print(a,b,x) # uses local a, use global b, use parameter x

a = 1
b = 10  # if you comment this line, g(11) will crash
g(11)

2 10 11


In [3]:
def f1(x):
    a = 1

    def f2(x):
        b = 2
        print(a,b,c,x)  # uses f1's a, local b, global c, parameter x
    
    f2(x)

a = 3
c = 5
f1(a)

1 2 5 3


Can f2 add 1 to the value of `a` defined in f1?

In [4]:
def f1(x):
    a = 1

    def f2(x):
        a = a + 1

    f2(x)

f1(a)

UnboundLocalError: local variable 'a' referenced before assignment

No, it cannot because it use the variable `a` defined in f2 on the right hand side in `a = a+1` **before** it declares it on the left hand side.

To do so, f2 must specify that the `a` it want to modify is **non local** which means use the more local. If you want to use the global variable `a`, say it with the **global** keyword:

In [5]:
def f1(x):
    a = 1

    def f2(x):
        nonlocal a
        a = a + 1  # now it knows we use a of f1
        print(a)
        
    def f3(x):
        global a
        a += 1
        print(a)
    
    print(a)
    f2(x)
    print(a)  # f2 has changed a of f1
    f3(x)
    print(a)  # but not f3

a = 10
f1(5)
print(a)  # changed by f3

1
2
2
11
2
11


**BUT ALL THIS IS BAD**

The right way to use variables is to use only local variables and if a function needs a variable from somewhere else, pass it as an argument. Each time you want to use `nonlocal` or `global` think twice before doing it.

# Scope

The scope of a variable is the set of lines which can call it. If a variable is declared in a function, it is all the lines from its declaration to the end of the function including all lines of nested functions. A parameter must be seen as a local variable alive from the first line of the function to its last one.

A variable is declared outside all functions and classes (and libraries) is a global variable which is *alive* from its declaration until the end of the program. It can be used by all function called after its declaration.

In [6]:
def f(x):
    k = 2
    
f(2)
print(k)   # k died at the end of f(x)!
print(x)   # x also died at the end of f(x)

NameError: name 'k' is not defined

# Scope in objects

An object is a variable like any other (in Python everything is an object), its scope is the same than any variable. All variable of an object die when their object die.

A class is a variable like any other (in Python everything is an object)...

However it is possible to **restrict the scope of a variable by making it private**. It is not technitly true in Python but it work the same by convention.

## Private variables

Private variables in a class are variables to be used only in the class. By being private they are protected from any outside modification.

Unlike C++, Python has not private methods or private variable. But a **convention** is to name variable to specify it is weakly private or strongly private :

* `_x` : a variable which starts with an underscore and does not finish with underscore it weakly private. It means Python will do nothing to protect but programmers are warned they should not use it outside the class.
* `__x` : a variable which starts with 2 underscores and does not finish with 2 underscores is strongly private. Python will rename it to `_classname__x` to protect it from modification however it can still be modify using its new name.

So if you want a private variable which can be used by child classes use weakly private `_`. If you want a private variable only used in its class and not in child classes, use strongly private `__`.

In [7]:
class A(object):
    def __init__(self):
        self._x = 0
        self.__y = 1
        
class B(A):
    def __init__(self):
        super().__init__()
        
    def print(self):
        print(self._x)
        print(self.__y)  # does not work
        
b = B()
print(b._x)  # it works but it is bad to call a private variable outside its class
b.print()

0
0


AttributeError: 'B' object has no attribute '_B__y'

## Private methods

Same applies with methods.

In [8]:
class A(object):
    def _f(self):
        print('A.f')
        
    def __g(self):
        print('A.g')
        
class B(A):
    def f(self):
        self._f()
        
    def g(self):
        self.__g()
        
b = B()
b.f()
b.g()

A.f


AttributeError: 'B' object has no attribute '_B__g'

{{ PreviousNext("02 Inheritance.ipynb", "04 View vs Copy.ipynb")}}